# Azure OpenAI Service API with OpenAI Python SDK

This notebook shows how to use the Azure OpenAI (AOAI) Service API with the [OpenAI Python SDK](https://github.com/openai/openai-python).

### Prerequisites

1. Setup for Azure Active Directory (AAD) authentication.
    * See [Setup to use AAD and test with CLI](setup_aad.md).
2. A Python environment setup with all the requirements.  
    * See the [setup_python_env.md](setup_python_env.md) page for instructions on setting up your environment.

### Configuring OpenAI SDK
The OpenAI SDK can be configured via _config_ file.  

 | SDK Variable | Description |
 | -- | -- |
 | `api_type` | API Type.  Use `azure_ad` for AAD authentication.  |
 | `api_version` | API Version.  You can use `2022-12-01`. |
 | `api_base` | Base URL for the API.  This is the endpoint URL for your team. |



In [1]:
import os
import openai
from dotenv import load_dotenv

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment_name, openai.api_base and openai.api_version.

In [2]:
# For Active Directory authentication
openai.api_type = "azure_ad"


# Set up Azure OpenAI
load_dotenv("credentials.env")

openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-03-15-preview"

In [3]:
print(openai.api_base)

https://openaiykus.openai.azure.com/


## Set up AAD authentication

`DefaultAzureCredential` can read your credentials from the environment after doing `az login`. 

In VS Code, you can use the Azure Account extension to log in with your Azure account.  If you are running this notebook in VS Code, be sure to restart VS Code after you do `az login`.

This article gives details on what identity `DefaultAzureCredential` uses: https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python

If you get an error similar to the following, you can try using `AzureCliCredential` instead of `DefaultAzureCredential`:

```
DefaultAzureCredential failed to retrieve a token from the included credentials. Attempted credentials:     EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured. Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot.this issue.     ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.     SharedTokenCacheCredential: Azure Active Directory error '(invalid_scope) AADSTS70011: The provided request must include a 'scope' input parameter. The provided value for the input parameter 'scope' is not valid. The scope https://cognitiveservices.azure.com is not valid. The scope format is invalid. Scope must be in a valid URI form <https://example/scope> or a valid Guid <guid/scope>. 
```



In [8]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DQPXDNV42 to authenticate.
Failed to authenticate 16b3c013-d300-468d-ac64-7eda0820b6d3 'Microsoft Non-Production' due to error 'AADSTS530004: AcceptCompliantDevice setting isn't configured for this organization. The admin needs to configure this setting to allow external users access to protected resources.
Trace ID: 8a46aad4-6192-45db-b4a8-d08051502a00
Correlation ID: 707efade-e09b-43f0-b1f4-6f5baad58255
Timestamp: 2023-08-15 14:58:57Z'
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
93d45a5c-5d75-4c08-9c21-3de55c2eb73b 'CentricaHDISandbox1'
bd5c6713-7399-4b31-be79-78f2d078e543 'Marks and Spencer'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "54b875cc-a81a-4914-8bfd-1a36bc7ddf4d",
    "isDefault": true,
    "managedByTenants": [],
    "name": "

In [13]:
from azure.identity import DefaultAzureCredential, AzureCliCredential #DefaultAzureCredential should work but you may need AzureCliCredential to make the authentication work
default_credential = AzureCliCredential()
#default_credential = DefaultAzureCredential()

## Define method to get a prompt completion using AAD authentication

The `refresh_token` function below is used to get a new token when the current token expires.  The `refresh_token` method is called by the `get_completion` to get the token if it is not already set or if the token has expired.

In [14]:
import datetime

token = None

def refresh_token():
    """Refresh AAD token"""
    global token
    # Check if Azure token is still valid
    if not token or datetime.datetime.fromtimestamp(token.expires_on) < datetime.datetime.now():
        token = default_credential.get_token("https://cognitiveservices.azure.com")


refresh_token()
# Set the API key to be your Bearer token (yes this could be optimizaed to not do this every time :D) 
openai.api_key = token.token


In [15]:
print(token)

AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2NvZ25pdGl2ZXNlcnZpY2VzLmF6dXJlLmNvbSIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0Ny8iLCJpYXQiOjE2OTIxMTEzMzksIm5iZiI6MTY5MjExMTMzOSwiZXhwIjoxNjkyMTE2ODEyLCJhY3IiOiIxIiwiYWlvIjoiQVZRQXEvOFVBQUFBOGRuTjFOVzhCSjN3RlU2NW5WUGord0tFRzNSQk1RY3N3Wjh1c3M4QzZ1K2Nqa09tdGlBNXlmelJWZ2JVNFNPSXJ2TlJyS2pZUmtScWU1d2Z0T2lRYjNlbHEwcDIvVVZIQm85R3BqQjcydGc9IiwiYW1yIjpbInB3ZCIsIm1mYSJdLCJhcHBpZCI6IjA0YjA3Nzk1LThkZGItNDYxYS1iYmVlLTAyZjllMWJmN2I0NiIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoiS2lsaW5jIiwiZ2l2ZW5fbmFtZSI6IlllbGl6IiwiZ3JvdXBzIjpbImU3YzQzZDAwLTQyMDAtNDRmZS04YmRiLWQ0OGZjNDg0MWUyYSIsIjIwNmFmMTAyLTQzZDAtNDEyMi04OTYxLWYxMmU0M2QwMjkxMyIsImZmOGJlODA1LWI5ZmQtNDBmNy1iNjFkLWM1NDZkMjNkOWI1ZCIsImNmZmJlYjA2LTkyYWMtNGU3OC1hNDQ1LTlkMzQ1MzJkYzFiYyIsIjI4NDYxOTBkLTA1ZGMtNDA0OC05MGJjLTdlMjM2ZjM0ZDg0YiIs

## Test the API

The payload parameters in the `get_completion` call are for illustration only and can be changed for your use case as described in the [reference](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference).

In [ ]:
# Example prompt 
prompt = "Once upon a time"

response = openai.Completion.create(
    deployment_id="gpt-35-turbo",
    prompt=prompt,
    temperature=0.7,
    max_tokens=300,
    top_p=0.5,
    stop=None
)

# printing the response
print(response)